In [1]:
import os
os.chdir("C:/Users/PraveenGupta/Downloads/Hackathon/SmartRecruits")

In [2]:
def string2date(str_date):
    try: #if the str_date has the format int-int-int
        int_date = map(lambda x:int(x),str_date.split('/'))
        return dt.date(int_date[2],int_date[0],int_date[1])
    except:
        print str_date
        return str_date

In [3]:
import pandas as pd
import numpy as np
import datetime as dt
import matplotlib.pyplot as plt
%matplotlib inline
from pprint import pprint
from datetime import datetime


from sklearn.preprocessing import LabelEncoder

In [4]:
# import data
df_train = pd.read_csv('Train_pjb2QcD.csv')
df_test = pd.read_csv('Test_wyCirpO.csv')

df_train['source']='train'
df_test['source']='test'

# Creating this 'data' dataframe so that all imputation is done at the same time.  train and test are split back from this
data = pd.concat([df_train,df_test],ignore_index=True)


In [5]:
# Prints the shape of all datasets
print "**size for:\n\tTrain:{0}\n\tTest:{1}\n\tOverall:{2}".format(df_train.shape,df_test.shape,data.shape)

**size for:
	Train:(9527, 24)
	Test:(5045, 23)
	Overall:(14572, 24)


In [6]:
# Renaming the columns for easy future usage
data.rename(columns={'Applicant_BirthDate':'Applicant_DoB','Manager_DOJ':'Manager_DoJ'}, inplace=True)

In [7]:
# Finding the number of missing values and unique values
miss_uniq_vals = []
for col in data.columns:
    miss_uniq_vals.append((100*len(data[data[col].isnull()])/float(data.shape[0]),len(pd.unique(data[col]))))
miss_uniq_vals = pd.DataFrame(miss_uniq_vals)
miss_uniq_vals.index = data.columns.tolist()
miss_uniq_vals.columns = ['perc_missing_val','num_uniq_val']
miss_uniq_vals

,perc_missing_val,num_uniq_val
Applicant_DoB,0.658798,7575
Applicant_City_PIN,0.898984,3757
Applicant_Gender,0.610760,3
Applicant_Marital_Status,0.706835,5
Applicant_Occupation,15.269009,6
Applicant_Qualification,0.892122,13
Application_Receipt_Date,0.000000,500
Business_Sourced,34.621191,3
ID,0.000000,14572
Manager_Business,10.341751,5315


In [8]:
# Impute the missing values with the mode
imputers = {}
for col in data.columns:
    if miss_uniq_vals.loc[col,'perc_missing_val']>0:
        # uncomment these lines to check the number of missing values before and after imputation
        #print col,'\t',df_train[df_train[col].isnull()].shape[0],'\t',
        most_freq = data[col].mode()[0]
        data[col].replace(np.nan,most_freq,inplace=True)
        imputers[col] = most_freq
        #print df_train[df_train[col].isnull()].shape[0]

In [9]:
# Calculate the ages of applicants, managers, manager experience and application receipt age
# doing this as the age cannot be processed by M/L algorithms..

dob = data['Applicant_DoB'].apply(lambda x:string2date(x))
dob=dob.apply(lambda x: x.year)
data['ApplicantAge']=(2009 - dob)

dob = data['Application_Receipt_Date'].apply(lambda x:string2date(x))
dob=dob.apply(lambda x: x.year)
data['ApplicantionReceiptAge']=(2009 - dob)

dob = data['Manager_DoJ'].apply(lambda x:string2date(x))
dob=dob.apply(lambda x: x.year)
data['ManagerExp']=(2009 - dob)         #should be decomposed into number of months

dob = data['Manager_DoB'].apply(lambda x:string2date(x))
dob=dob.apply(lambda x: x.year)
data['ManagerAge']=(2009 - dob)


In [10]:
# One Hot Encoding
data_types = {'dummies':['Office_PIN','Applicant_City_PIN','Applicant_Gender','Applicant_Marital_Status',
                   'Applicant_Occupation','Applicant_Qualification','Manager_Status','Manager_Gender',
                   'Manager_Joining_Designation','Manager_Current_Designation']}

for col in data.columns[:-1]:
    if col in data_types['dummies']:
        label_encoders = LabelEncoder()
        data[col]=label_encoders.fit_transform(data[col])

# Create Dummies for all the categorical columns 
data = pd.get_dummies(data, columns = data_types['dummies'])

In [11]:
# splitting back the data into train and test
df_train=data.loc[data['source']=='train']
df_test=data.loc[data['source']=='test']

#Drop unnecessary columns
df_test.drop(['source','Business_Sourced'],axis=1,inplace=True)
df_train.drop(['source'],axis=1,inplace=True)

c:\Anaconda2\lib\site-packages\ipykernel\__main__.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
c:\Anaconda2\lib\site-packages\ipykernel\__main__.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [12]:
#Export files as modified versions for future simple usage
df_train.to_csv("/Users/PraveenGupta/Downloads/Hackathon/SmartRecruits/Final/train_modified.csv",index=False)
df_test.to_csv("/Users/PraveenGupta/Downloads/Hackathon/SmartRecruits/Final/test_modified.csv",index=False)

In [13]:
import xgboost as xgb
from xgboost.sklearn import XGBClassifier
from sklearn import cross_validation, metrics

target = 'Business_Sourced'
IDCol = ['ID']

#Function for XGBoost
def xGBmodelfit(alg,dtrain,dtest,predictors,filename,useTrainCV=True, cv_folds=5,early_stopping_rounds=200):
    print datetime.now().time()
    if useTrainCV:
        xgb_param=alg.get_xgb_params()
        xgtrain = xgb.DMatrix(dtrain[predictors].values,label = dtrain[target].values)
        cvresult = xgb.cv(xgb_param,xgtrain,num_boost_round=alg.get_params()['n_estimators'],
            nfold=cv_folds,metrics='auc',early_stopping_rounds=early_stopping_rounds)#,show_progress=False)
        alg.set_params(n_estimators=cvresult.shape[0])#,booster='gbtree')
    
    bestNoOfRounds = cvresult['test-auc-mean'].argmax()
    bestAnsForThisSetting =  cvresult.ix[bestNoOfRounds]
    bestModelEval = bestAnsForThisSetting[0]
    print "Best Num of Rounds %.7g" %bestNoOfRounds
    print "Best Model Eval (Test-AUC) %.7g" %bestModelEval
    
    alg.fit(dtrain[predictors],dtrain[target],eval_metric='auc')#,eta=0.0001)
    
#   Predicting training data
    dtrain_predictions = alg.predict(dtrain[predictors])
    dtrain_predprob=alg.predict_proba(dtrain[predictors])[:,1]       
    
    print "\nModel Report"
    print "Accuracy : %.4g" %metrics.accuracy_score(dtrain[target],dtrain_predictions)
    print "AUC Score (train) : %f" %metrics.roc_auc_score(dtrain[target],dtrain_predprob)
    
#   Feature importance plot
    feat_imp = pd.Series(alg.booster().get_fscore()).sort_values(ascending=False)
    feat_imp.plot(kind='bar',title = 'Feature Importance')
    plt.ylabel('Feature Importance score')
    
    #Predict on testing data
    dtest[target]=alg.predict_proba(dtest[predictors])[:,1]
    
    #Export submission file
    IDCol.append(target)
    submission = pd.DataFrame({x : dtest[x] for x in IDCol})
    submission.to_csv(filename,index=False)
    
    print datetime.now().time()

In [14]:
%matplotlib qt
extra = ['Applicant_DoB','Application_Receipt_Date','Manager_DoJ','Manager_DoB']

predictors = [x for x in df_train.columns if x not in [target]+IDCol+extra]

xgb1 = XGBClassifier(
objective = 'binary:logistic',
learning_rate=0.05,
n_estimators = 200,
max_depth=5,
min_child_weight=5,
subsample=0.8,
colsample_bytree=0.5,
nthread=4,
seed=27)

xGBmodelfit(xgb1,df_train,df_test,predictors,'Final/submission_xgBoost.csv')

18:21:17.722000
Best Num of Rounds 196
Best Model Eval (Test-AUC) 0.6323636

Model Report
Accuracy : 0.6971
AUC Score (train) : 0.741543
18:27:10.907000


c:\Anaconda2\lib\site-packages\ipykernel\__main__.py:40: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
